In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#sleep report 신청한 유저
aos_user = pd.read_json('/content/drive/MyDrive/bq-results-20231030-052807-1698643696287/bq-results-20231030-052807-1698643696287.json', lines = True)

In [ ]:
aos_user

,0,1,2,3,4,5,6,7,8,9,...,43813,43814,43815,43816,43817,43818,43819,43820,43821,43822
createdDate,1666787882431.0,1667341690093.0,1666268977929.0,1658800886269.0,1667892630153.0,1663600336646.0,1679000823.0,1654771505209.0,1665935675964.0,1661507202299.0,...,1673361962955.0,1671648167514.0,1676635951.0,1677677596.0,1663034197255.0,1678653628.0,1678658247.0,1678714060.0,1663041256522.0,1666491913205.0
totalSleep,1.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,...,0.0,1.0,2.0,176.0,24.0,0.0,0.0,1.0,0.0,9.0
rating,0.0,0.0,0.0,NaN,0.0,0.0,NaN,6.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
modifiedDate,2022-10-26 12:38:02.431 UTC,2022-11-01 22:28:10.093 UTC,2022-10-20 12:29:37.929 UTC,2022-07-26 02:01:26.269 UTC,2022-11-08 07:30:30.153 UTC,2022-09-19 15:12:16.646 UTC,2023-03-16 21:07:03.099 UTC,2022-06-14 15:00:00.605 UTC,2022-10-16 15:54:35.964 UTC,2022-08-26 09:46:42.299 UTC,...,2023-01-10 14:46:02.955 UTC,2022-12-21 18:42:47.514 UTC,2023-02-17 12:12:31.986 UTC,2023-03-01 13:33:16.589 UTC,2022-09-13 01:56:37.255 UTC,2023-03-12 20:40:28.482 UTC,2023-03-12 21:57:27.806 UTC,2023-03-13 13:27:40.475 UTC,2022-12-03 15:00:00.145 UTC,2022-10-23 02:25:13.205 UTC
purchaseNum,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
userName,"{'string': '무민', 'provided': 'string'}","{'string': '달롯', 'provided': 'string'}","{'string': '송우철', 'provided': 'string'}","{'string': 'EDEN', 'provided': 'string'}","{'string': 'TV박정환', 'provided': 'string'}","{'string': 'Chloe Kim', 'provided': 'string'}","{'string': '김성곤', 'provided': 'string'}","{'string': '초코공주', 'provided': 'string'}","{'string': '양원준', 'provided': 'string'}","{'string': 'lily lily', 'provided': 'string'}",...,"{'string': 'Sunyoung Park', 'provided': 'string'}","{'string': 'JO JO', 'provided': 'string'}","{'string': '아비지율', 'provided': 'string'}","{'string': '2갱갱', 'provided': 'string'}","{'string': '조미건', 'provided': 'string'}","{'string': '억소리', 'provided': 'string'}","{'string': '김봉현', 'provided': 'string'}","{'string': '우용환', 'provided': 'string'}",NaN,"{'string': '공도현', 'provided': 'string'}"
nds,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,0.0
email,"{'string': 'moomin4863@gmail.com', 'provided':...","{'string': 'dydtjgo4026@gmail.com', 'provided'...","{'string': 'cnsqo0914@gmail.com', 'provided': ...","{'string': 'edennam@gmail.com', 'provided': 's...","{'string': 'itvparkf1pjw@gmail.com', 'provided...","{'string': 'jjiyo1004@gmail.com', 'provided': ...","{'string': 'gonskim3@gmail.com', 'provided': '...","{'string': 'sallyso4999@gmail.com', 'provided'...","{'string': '1junineer@gmail.com', 'provided': ...","{'string': 'kyungsun.shin@gmail.com', 'provide...",...,"{'string': 'sunyoung514@gmail.com', 'provided'...","{'string': 'ghks5527@gmail.com', 'provided': '...","{'string': 'kwon20011153@gmail.com', 'provided...","{'string': 'ekl13579@gmail.com', 'provided': '...","{'string': 'migeoncho07@gmail.com', 'provided'...","{'string': 'naa7801@gmail.com', 'provided': 's...","{'string': 'joeun9771@gmail.com', 'provided': ...","{'string': 'abc19281928@gmail.com', 'provided'...","{'string': 'max300800@gmail.com', 'provided': ...","{'string': 'a53826505@gmail.com', 'provided': ..."
coin,"{'integer': '0', 'provided': 'integer'}","{'integer': '1', 'provided': 'integer'}","{'integer': '1', 'provided': 'integer'}","{'integer': '1', 'provided': 'integer'}","{'integer': '0', 'provided': 'integer'}","{'integer': '0', 'provided': 'integer'}","{'integer': '1', 'provided': 'integer'}","{'integer': '1', 'provided': 'integer'}","{'integer': '0', 'provided': 'integer'}","{'integer': '0', 'provided': 'integer'}",...,"{'integer': '1', 'provided': 'integer'}","{'integer': '0', 'provided': 'integer'}","{'integer': '0', 'provided': 'integer'}","{'integer': '-175', 'provided': 'integer'}","{'integer': '22', 'provided': 'integer'}","{'integer': '1', 'provided': 'integer'}","{'integer': '1', 'provided': 'integer'}","{'integer': '0', 'provided'

In [ ]:
aos_user = aos_user.transpose()

In [ ]:
aos_user_report = aos_user[aos_user['sleepReportYes'] == 1.0]
info = aos_user_report[['uid', 'nickname', 'phoneNumber']]
info.columns = ['uid', 'username', 'phone_number']


In [ ]:
aos_user_status = pd.read_json("/content/drive/MyDrive/bq-results-20231030-052657-1698643625364/bq-results-20231030-052657-1698643625364.json", lines=True)
aos_sleeptime = aos_user_status.copy()


In [ ]:
aos_sleeptime.columns

Index(['uid', 'createdDate', 'sleepSeconds', 'wakeTimeSeconds', 'statusOthers',
       'statusFeels', 'startedAt', 'statusActs', 'monauralId', 'soundVolume',
       'recommendVolume', 'endedAt', 'selectedLabelList', 'rating',
       'sleepComment', 'statusList', 'meditation', '__key__', '__error__',
       '__has_error__', 'selectedLabels', 'yesMeditation', 'soundId',
       'audible', 'asmrId', 'tipId', 'selectedStatus', 'asmrTitle',
       'monauralVol', 'asmrVol', 'meditationId'],
      dtype='object')

In [ ]:
aos_sleeptime = aos_sleeptime[aos_sleeptime['createdDate']>='2023-10-01']
aos_sleeptime = aos_sleeptime[aos_sleeptime['startedAt']>1694995200000]
aos_sleeptime = aos_sleeptime[['createdDate', 'sleepSeconds', 'wakeTimeSeconds','startedAt', 'uid','sleepComment', 'statusActs', 'statusFeels' ]]

aos_sleeptime = aos_sleeptime.dropna(subset=['uid', 'startedAt', 'wakeTimeSeconds'])


In [ ]:
## epoch time을 2023-09-18 형식으로 바꾸기

## second to date
from datetime import datetime
import pandas as pd

# Epoch 시간을 datetime 객체로 변환하는 함수 정의
def convert_epoch_to_datetime(epoch_time):
    epoch_time /= 1000  # 밀리초를 초로 변환
    return datetime.utcfromtimestamp(epoch_time)

# "startedAt" 열의 값을 변환
aos_sleeptime['startedAt'] = aos_sleeptime['startedAt'].apply(convert_epoch_to_datetime)
#aos_sleeptime['wakeTimeSeconds'] = aos_sleeptime['wakeTimeSeconds'].apply(convert_epoch_to_datetime)

from datetime import datetime
import pandas as pd

# Epoch 시간을 datetime 객체로 변환하는 함수 정의
def convert_epoch_to_datetime(epoch_time):
    #epoch_time /= 1000  # 밀리초를 초로 변환
    return datetime.utcfromtimestamp(epoch_time)

# "startedAt" 열의 값을 변환
#aos_sleeptime['startedAt'] = aos_sleeptime['startedAt'].apply(convert_epoch_to_datetime)
aos_sleeptime['wakeTimeSeconds'] = aos_sleeptime['wakeTimeSeconds'].apply(convert_epoch_to_datetime)

In [ ]:
## 맞는 형식으로 바꾸기

aos_sleeptime['startedAt'] = aos_sleeptime['startedAt'].dt.strftime('%Y-%m-%d %H:%M:%S')
aos_sleeptime['wakeTimeSeconds'] = aos_sleeptime['wakeTimeSeconds'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
## datetime, utc+9로 바꾸기

import pandas as pd
import pytz

# 문자열을 datetime 객체로 변환하는 함수 정의
def convert_str_to_datetime(dt_str):
    return pd.to_datetime(dt_str)

# 문자열을 datetime 객체로 변환
aos_sleeptime['startedAt'] = aos_sleeptime['startedAt'].apply(convert_str_to_datetime)
aos_sleeptime['wakeTimeSeconds'] = aos_sleeptime['wakeTimeSeconds'].apply(convert_str_to_datetime)

# UTC를 UTC+9로 조정하는 함수 정의
def adjust_to_seoul_timezone(dt):
    utc_timezone = pytz.utc
    seoul_timezone = pytz.timezone('Asia/Seoul')

    dt_utc = utc_timezone.localize(dt)
    dt_seoul = dt_utc.astimezone(seoul_timezone)

    return dt_seoul

# "wakeTimeSeconds"와 "startedAt" 열의 값을 조정
aos_sleeptime['wakeTimeSeconds'] = aos_sleeptime['wakeTimeSeconds'].apply(adjust_to_seoul_timezone)
aos_sleeptime['startedAt'] = aos_sleeptime['startedAt'].apply(adjust_to_seoul_timezone)

aos_sleeptime['wakeTimeSeconds'] = aos_sleeptime['wakeTimeSeconds'].dt.tz_localize(None)
aos_sleeptime['startedAt'] = aos_sleeptime['startedAt'].dt.tz_localize(None)


In [ ]:
aos_sleeptime_091824 = aos_sleeptime[(aos_sleeptime['startedAt']>='2023-10-23 21:00:00') & (aos_sleeptime['startedAt']<='2023-10-30 06:00:00') &(aos_sleeptime['wakeTimeSeconds']<='2023-10-30 13:00:00')& (aos_sleeptime['wakeTimeSeconds']>='2023-10-24')]

In [ ]:
##요일별 취침/기상시간

import pandas as pd


# 요일 정보를 갱신하는 함수. 9.18월 ~ 9.25 월까지라서 월요일 취침시간이 중복될 수도 있음. (수면 시간은 월~일)
# 월요일 취침 - 화요일 기상 : 월요일 수면 시간, 다음주 월요일 취침 - 다음주 월요일 기상 : 일요일 수면 시간
# 그래서 취침 정보는 월 ~ 월까지 필요함. 기상 정보는 화~월. 수면 시간은 월~일.

def update_weekday(row):
    if row['startedAt'].date() >= pd.to_datetime('2023-10-30'):
        return 'next_mon'
    return row['start_weekday']

# 요일 정보 및 시간만 추출
weekday_kor = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'next_mon']
weekday_kor2 = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
aos_sleeptime_091824['start_weekday'] = aos_sleeptime_091824['startedAt'].dt.strftime('%a').str.lower()
aos_sleeptime_091824['start_weekday'] = aos_sleeptime_091824.apply(update_weekday, axis=1)

aos_sleeptime_091824['wakeup_weekday'] = aos_sleeptime_091824['wakeTimeSeconds'].dt.strftime('%a').str.lower()

# Pivot tables 생성
sleep_pivot = aos_sleeptime_091824.pivot_table(index='uid', columns='start_weekday', values='startedAt', aggfunc='first').applymap(lambda x: x.time() if pd.notnull(x) else x)[weekday_kor]
wake_pivot = aos_sleeptime_091824.pivot_table(index='uid', columns='wakeup_weekday', values='wakeTimeSeconds', aggfunc='first').applymap(lambda x: x.time() if pd.notnull(x) else x)[weekday_kor2]



<ipython-input-30-110677ee6e9d>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aos_sleeptime_091824['start_weekday'] = aos_sleeptime_091824['startedAt'].dt.strftime('%a').str.lower()
<ipython-input-30-110677ee6e9d>:11: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if row['startedAt'].date() >= pd.to_datetime('2023-10-30'):
<ipython-input-30-110677ee6e9d>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [ ]:
## 수면 시간 계산
def calculate_sleep_duration(row):
    # 임의의 날짜를 사용하여 시간만 있는 객체를 datetime.datetime으로 변환
    wake_time = pd.Timestamp("2000-01-01 " + str(row['wakeTimeSeconds'].time()))
    started_time = pd.Timestamp("2000-01-01 " + str(row['startedAt'].time()))
    if row['start_weekday'] == 'next_mon':
          row['start_weekday'] = 'mon'
    if row['start_weekday'] == row['wakeup_weekday']:
      duration = wake_time - started_time
      previous_day_dict = {
            'mon': 'sun', 'tue': 'mon', 'wed': 'tue', 'thu': 'wed', 'fri': 'thu', 'sat': 'fri', 'sun': 'sat'
        }
      row['start_weekday'] = previous_day_dict[row['start_weekday']]
    else:
      modified_wake_time = wake_time + pd.Timedelta(hours=24)
      duration = modified_wake_time - started_time
      #print(modified_wake_time, duration)
      #print(f"wake_time {modified_wake_time}, start {started_time}, duration {duration}")

    hours = duration.total_seconds() // 3600
    minutes = (duration.total_seconds() % 3600) // 60
    return row['start_weekday'], f"{int(hours)}:{int(minutes):02d}"



# 새로운 열 생성
aos_sleeptime_091824[['modified_start_weekday', 'sleep_duration']] = aos_sleeptime_091824.apply(calculate_sleep_duration, axis=1, result_type='expand')

# 문자열 시간을 소수점 형태의 시간으로 변환하는 함수
def convert_str_time_to_decimal(time_str):
    hours, minutes = map(int, time_str.split(":"))
    return round(hours + minutes/60, 1) # 소수점 첫째 자리까지 반올림

aos_sleeptime_091824['decimal_sleep_hours'] = aos_sleeptime_091824['sleep_duration'].apply(convert_str_time_to_decimal)

# Pivot table
sleep_duration_pivot = aos_sleeptime_091824.pivot_table(index='uid', columns='modified_start_weekday', values='decimal_sleep_hours', aggfunc='mean')[weekday_kor2]


<ipython-input-31-fd8714bde1ad>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aos_sleeptime_091824[['modified_start_weekday', 'sleep_duration']] = aos_sleeptime_091824.apply(calculate_sleep_duration, axis=1, result_type='expand')
<ipython-input-31-fd8714bde1ad>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aos_sleeptime_091824[['modified_start_weekday', 'sleep_duration']] = aos_sleeptime_091824.apply(calculate_sleep_duration, axis=1, result_type='expand')
<ipython-input-31-fd8714bde1ad>:34: SettingW

In [ ]:
sleep_duration_pivot.columns = ['mon_sleep_hours', 'tue_sleep_hours', 'wed_sleep_hours', 'thu_sleep_hours', 'fri_sleep_hours', 'sat_sleep_hours', 'sun_sleep_hours']

In [ ]:
# 평균 수면 시간
sleep_hour_average = sleep_duration_pivot.mean(axis=1).round(1).reset_index()
sleep_hour_average.columns = ['uid', 'average_sleep_hours']

In [ ]:
aos_sleephour_excel = pd.merge(info, sleep_hour_average, on = 'uid', how = 'left')

In [ ]:
wake_pivot.columns = (['mon_waketime', 'tue_waketime', 'wed_waketime', 'thu_waketime', 'fri_waketime',
       'sat_waketime', 'sun_waketime'])

In [ ]:
## 평균 취침 / 기상 시간

import pandas as pd

def time_to_str(hour_float):
    hour = int(hour_float)
    minute = int((hour_float - hour) * 60)
    second = int((((hour_float - hour) * 60) - minute) * 60)
    return f"{hour:02d}:{minute:02d}:{second:02d}"

def average_times(df):
    # 문자열을 datetime으로 변환
    df['startedAt'] = pd.to_datetime(df['startedAt'])
    df['wakeTimeSeconds'] = pd.to_datetime(df['wakeTimeSeconds'])

    # 취침시간이 오전이면 다음날로 처리
    df.loc[df['startedAt'].dt.hour < 12, 'startedAt'] += pd.Timedelta(hours=24)

    # 시간만 추출. 12시 이후의 취침은 24를 더해서 다음날로 처리
    df['average_bedtime'] = df['startedAt'].dt.hour + df['startedAt'].dt.minute / 60.0 + df['startedAt'].dt.second / 3600.0
    df['average_bedtime'] = df['average_bedtime'].apply(lambda x: x if x > 12 else x + 24)

    df['average_waketime'] = df['wakeTimeSeconds'].dt.hour + df['wakeTimeSeconds'].dt.minute / 60.0 + df['wakeTimeSeconds'].dt.second / 3600.0

    # uid로 그룹화하여 평균 시간 계산
    result = df.groupby('uid')[['average_bedtime', 'average_waketime']].mean().reset_index()

    # 시간 값을 HH:MM:SS 형식의 문자열로 변환
    result['average_bedtime'] = result['average_bedtime'].apply(lambda x: time_to_str(x % 24))  # 24로 나눈 나머지로 시간 표현
    result['average_waketime'] = result['average_waketime'].apply(time_to_str)

    return result

avg_df = average_times(aos_sleeptime_091824)
print(avg_df)


                              uid average_bedtime average_waketime
0    034WnkRLrKgC3qGzisY6SBDtztG2        02:12:34         07:57:29
1    07ungtuKLpYP3vYIgIww4Q66RcU2        01:18:48         08:24:59
2    0DbVhIqMXiOo6wLjzq8WY2m4FJl2        23:31:11         05:33:20
3    0bVrNI3muGgNOcEYrZIQKc1S4jo1        05:32:30         07:10:00
4    0suvqno3VGX6hpMzThtSQJHq7YI3        01:27:51         06:57:29
..                            ...             ...              ...
949  z6iqkHbDtrSj22bE32ZvAev9eAD3        04:50:33         08:00:00
950  zPgR0rPu71UEps0HLVHMvmhyWbJ2        03:49:08         07:09:29
951  zlsG4G8f0eR3hehUuMw6Lmdzcq13        01:50:21         08:13:44
952  zoeoQhF2JvgGxHejrbmLBTiVOcJ2        00:46:33         06:45:00
953  zqs8wesFhrfArR9GaUqjNlASxtZ2        00:00:30         06:03:59

[954 rows x 3 columns]


<ipython-input-36-c6987a6d0e06>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['startedAt'] = pd.to_datetime(df['startedAt'])
<ipython-input-36-c6987a6d0e06>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wakeTimeSeconds'] = pd.to_datetime(df['wakeTimeSeconds'])
<ipython-input-36-c6987a6d0e06>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [ ]:
aos_sleephour_excel = pd.merge(aos_sleephour_excel, avg_df, on='uid', how='left')

In [ ]:
## wake time pivot을 이용한 수면 일정도 후후

import pandas as pd


# 문자열 시간을 datetime.time으로 변환합니다.
wake_pivot = wake_pivot.applymap(lambda x: pd.to_datetime(x, format='%H:%M:%S', errors='coerce').time() if pd.notna(x) else None)

# 함수 정의: 새로운 최소/최대 계산 로직
def calc_min_max(series):
    times = series.dropna().apply(lambda x: x if x is not pd.NaT else None).dropna()  # NaT 제거
    min_time = min(times, default=None)
    max_time = max(times, default=None)

    return min_time, max_time

# 결과를 계산하고 데이터 프레임에 적용합니다.
results = wake_pivot.apply(calc_min_max, axis=1, result_type='expand')
results.columns = ['earliest_wakeup', 'latest_wakeup']


In [ ]:
wake_pattern = pd.merge(wake_pivot, results, on = 'uid', how ='left')

#time_pivot.columns = ['mon_sleep_hours', 'tue_sleep_hours', 'wed_sleep_hours', 'thu_sleep_hours', 'fri_sleep_hours', 'sat_sleep_hours', 'sun_sleep_hours']


In [ ]:
## sleep time pivot을 이용한 수면 일정도 후후

import pandas as pd


# 문자열 시간을 datetime.time으로 변환합니다.
sleep_pivot = sleep_pivot.applymap(lambda x: pd.to_datetime(x, format='%H:%M:%S', errors='coerce').time() if pd.notna(x) else None)

# 함수 정의: 새로운 최소/최대 계산 로직
def calc_min_max(series):
    times = series.dropna()

    after_midnight = [t for t in times if t.hour < 12]
    before_midnight = [t for t in times if t.hour >= 12 and t.hour < 24]



    # 24시 이전과 이후의 시간이 모두 있을 경우
    if before_midnight and after_midnight:
        min_time = min(before_midnight, default=None)
        max_time = max(after_midnight, default=None)
    # 모든 시간이 24시 이전일 경우
    elif all(t.hour < 24 for t in times):
        min_time = min(times, default=None)
        max_time = max(times, default=None)
    # 24시 이전의 시간만 있을 경우
    elif before_midnight:
        min_time = min(before_midnight, default=None)
        max_time = max(before_midnight, default=None)
    # 24시 이후의 시간만 있을 경우
    else:
        min_time = min(after_midnight, default=None)
        max_time = max(after_midnight, default=None)

    return min_time, max_time



# 결과를 계산하고 데이터 프레임에 적용합니다.
sleep_results = sleep_pivot.apply(calc_min_max, axis=1, result_type='expand')
sleep_results.columns = ['earliest_sleep', 'latest_sleep']


In [ ]:
## latest - early

from datetime import datetime, timedelta

# 시간 차이 계산 함수
def calculate_time_difference(early, latest):
    FMT = '%H:%M:%S'
    tdelta = datetime.strptime(str(latest), FMT) - datetime.strptime(str(early), FMT)

    # 음수로 나온 경우 24시간 더하기
    if tdelta.days < 0:
        tdelta += timedelta(days=1)

    return tdelta

sleep_results['wake_time_difference'] = sleep_results.apply(lambda row: calculate_time_difference(row['earliest_sleep'], row['latest_sleep']), axis=1)
sleep_results['sleep_time_difference'] = results.apply(lambda row: calculate_time_difference(row['earliest_wakeup'], row['latest_wakeup']), axis=1)

def calculate_penalty(time_difference):
    # 시간 차이를 분으로 변환
    minutes_diff = time_difference.seconds // 60

    # 10분마다 2% 패널티
    penalty = (minutes_diff // 10) * 2

    # 최대 패널티는 30
    return min(penalty, 30)

#최종 패널티
sleep_results['sleep_pattern'] = 100 - (sleep_results['wake_time_difference'].apply(calculate_penalty) + sleep_results['sleep_time_difference'].apply(calculate_penalty))
sleep_results.reset_index(inplace = True)


In [ ]:
penalty = sleep_results[['uid', 'sleep_pattern']]

In [ ]:
aos_sleephour_excel = pd.merge(aos_sleephour_excel, penalty, on='uid', how='left')

In [ ]:
aos_sleephour_excel = pd.merge(aos_sleephour_excel, sleep_duration_pivot, on = 'uid', how = 'left')


In [ ]:
#aos_sleephour_excel = pd.merge(aos_sleephour_excel, sleep_pivot, on = 'uid', how = 'left')


In [ ]:
sleep_pivot = sleep_pivot.reset_index()
sleep_pivot[sleep_pivot['uid'] == 'H113sVpi3BZz4Ukb4zILzzdly4C3']

start_weekday,index,uid,mon,tue,wed,thu,fri,sat,sun,next_mon
92,92,H113sVpi3BZz4Ukb4zILzzdly4C3,None,00:23:23,00:15:13,00:13:31,00:10:32,00:50:20,None,None


In [ ]:
aos_sleephour_excel

,uid,username,phone_number,average_sleep_hours,average_bedtime,average_waketime,sleep_pattern,mon_sleep_hours,tue_sleep_hours,wed_sleep_hours,thu_sleep_hours,fri_sleep_hours,sat_sleep_hours,sun_sleep_hours
0,1V5AlJ75a4URWguw9sQDnuiewzT2,배예령,01099629292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1ZQHxyIKmMXNvNbpqmJYwgxkSGW2,유리,01066266591,7.2,00:26:45,07:35:59,92.0,7.5,6.9,NaN,NaN,NaN,NaN,NaN
2,2KorMkK5DbUG2GIkxNgbthI5gCm2,진저브래드맨,01024421975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2RHmSMvrnAgWtZExO5dneSXtfYp1,강한버니77,01033849973,6.0,17:06:38,23:10:00,100.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN
4,2jcvWeMBsuaRxnLE9PLFSOOZSZ13,한길,01093839440,6.6,03:32:49,10:11:40,86.0,6.7,7.0,7.0,6.5,6.5,6.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,wbplTesO1ZcDFoPcnvggcUgmNMb2,숙면중,01094235165,7.4,22:57:14,06:22:08,66.0,7.5,7.4,7.7,7.6,7.8,7.6,6.20
204,wscZR0tz5pSEsdZuoUvjtMI9ger1,WiseBunny76,1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,xp25BZKC8lgubCcWesspxbwA0N93,오둥쓰,01088138431,5.9,00:31:01,06:31:15,40.0,6.1,6.0,6.1,5.3,NaN,6.0,6.15
206,yUM4rXgTQLZRU5HuoITE3MWxzln2,예준님,01049662908,4.8,01:42:58,06:32:29,74.0,NaN,NaN,6.0,NaN,3.6,NaN,NaN


In [ ]:
aos_sleephour_excel = pd.merge(aos_sleephour_excel, wake_pivot, on = 'uid', how = 'left')

In [ ]:
aos_sleephour_excel

,uid,username,phone_number,average_sleep_hours,average_bedtime,average_waketime,sleep_pattern,mon_sleep_hours,tue_sleep_hours,wed_sleep_hours,...,fri_sleep_hours,sat_sleep_hours,sun_sleep_hours,mon_waketime,tue_waketime,wed_waketime,thu_waketime,fri_waketime,sat_waketime,sun_waketime
0,1V5AlJ75a4URWguw9sQDnuiewzT2,배예령,01099629292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1ZQHxyIKmMXNvNbpqmJYwgxkSGW2,유리,01066266591,7.2,00:26:45,07:35:59,92.0,7.5,6.9,NaN,...,NaN,NaN,NaN,None,07:30:00,07:45:00,None,None,None,None
2,2KorMkK5DbUG2GIkxNgbthI5gCm2,진저브래드맨,01024421975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2RHmSMvrnAgWtZExO5dneSXtfYp1,강한버니77,01033849973,6.0,17:06:38,23:10:00,100.0,NaN,6.0,NaN,...,NaN,NaN,NaN,None,None,23:10:00,None,None,None,None
4,2jcvWeMBsuaRxnLE9PLFSOOZSZ13,한길,01093839440,6.6,03:32:49,10:11:40,86.0,6.7,7.0,7.0,...,6.5,6.1,NaN,None,10:00:00,10:35:00,10:45:00,10:00:00,10:00:00,09:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,wbplTesO1ZcDFoPcnvggcUgmNMb2,숙면중,01094235165,7.4,22:57:14,06:22:08,66.0,7.5,7.4,7.7,...,7.8,7.6,6.20,06:40:00,06:00:00,06:45:00,06:30:00,06:40:00,06:00:00,06:00:00
204,wscZR0tz5pSEsdZuoUvjtMI9ger1,WiseBunny76,1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,xp25BZKC8lgubCcWesspxbwA0N93,오둥쓰,01088138431,5.9,00:31:01,06:31:15,40.0,6.1,6.0,6.1,...,NaN,6.0,6.15,08:55:00,04:30:00,05:00:00,05:10:00,05:10:00,None,09:35:00
206,yUM4rXgTQLZRU5HuoITE3MWxzln2,예준님,01049662908,4.8,01:42:58,06:32:29,74.0,NaN,NaN,6.0,...,3.6,NaN,NaN,None,None,None,07:05:00,None,06:00:00,None


In [ ]:

# 수면 리포트 실패
aos_fail = aos_sleephour_excel[aos_sleephour_excel.iloc[:, 7:14].notna().sum(axis=1) < 4]
aos_fail = aos_fail[['uid', 'phone_number', 'username']]
aos_fail.to_csv("aos_fail.csv", encoding = 'utf-8-sig', index = False)

# 수면 리포트 성공
aos_success = aos_sleephour_excel[aos_sleephour_excel.iloc[:, 7:14].notna().sum(axis=1) >= 4]
aos_success = aos_success[['uid', 'phone_number', 'username']]
aos_success.to_csv("aos_success.csv", encoding = 'utf-8-sig', index = False)


In [ ]:
aos_sleephour_excel = aos_sleephour_excel[aos_sleephour_excel.iloc[:, 7:14].notna().sum(axis=1) >= 4]

In [ ]:
aos_sleeptime

,createdDate,sleepSeconds,wakeTimeSeconds,startedAt,uid,sleepComment,statusActs,statusFeels
1369,2023-10-04 18:22:30.182 UTC,"{'integer': '16200', 'provided': 'integer'}",2023-10-05 08:20:00,2023-10-05 03:22:39,kakao:2764058930,NaN,[],[]
1385,2023-09-25 17:44:44.754 UTC,"{'integer': '16200', 'provided': 'integer'}",2023-09-26 07:20:00,2023-09-26 02:44:51,kakao:2764058930,NaN,[alcohol],[]
1390,2023-10-05 17:26:37.001 UTC,"{'integer': '16200', 'provided': 'integer'}",2023-10-06 07:30:00,2023-10-06 02:26:46,kakao:2764058930,NaN,[nicotine],[]
1421,2023-10-02 18:13:29.76 UTC,"{'integer': '16200', 'provided': 'integer'}",2023-10-03 06:10:00,2023-10-03 03:13:44,kakao:2765753333,NaN,[],[tired]
1487,2023-09-20 17:25:22.751 UTC,"{'integer': '16200', 'provided': 'integer'}",2023-09-21 08:00:00,2023-09-21 02:25:47,kakao:2770126269,NaN,[phone],"[stressed, tired]"
...,...,...,...,...,...,...,...,...
101435,2023-09-26 11:33:28.883 UTC,"{'integer': '16200', 'provided': 'integer'}",2023-09-27 02:35:00,2023-09-26 20:33:40,zcKhOrGj44RZRkf2QHfR3spcC1F3,,[],[]
101436,2023-10-03 02:53:56.879 UTC,"{'integer': '16200', 'provided': 'integer'}",2023-10-03 17:55:00,2023-10-03 11:54:04,zcKhOrGj44RZRkf2QHfR3spcC1F3,,[],[]
101437,2023-09-26 09:05:32.516 UTC,"{'integer': '16200', 'provided': 'integer'}",2023-09-27 00:05:00,2023-09-26 18:05:47,zcKhOrGj44RZRkf2QHfR3spcC1F3,,[],[]
101438,2023-09-28 16:36:54.006 UTC,"{'integer': '16200', 'provided': 'integer'}",2023-09-29 07:40:00,2023-09-29 01:37:00,zcKhOrGj44RZRkf2QHfR3spcC1F3,,[],[]


In [ ]:
# statusActs 열의 값들을 uid 별로 합칩니다.
grouped_acts = aos_sleeptime.groupby('uid')['statusActs'].sum().reset_index()

# statusFeels 열의 값들을 uid 별로 합칩니다.
grouped_feels = aos_sleeptime.groupby('uid')['statusFeels'].sum().reset_index()

# 위에서 얻은 결과를 병합하여 하나의 데이터프레임으로 만듭니다.
merged = grouped_acts.merge(grouped_feels, on='uid')


In [ ]:
aos_sleeptime[['statusActs']]

,statusActs
1407,[nicotine]
1417,[]
1461,[]
1518,[]
1523,[]
...,...
103481,[]
103482,[nicotine]
103483,[]
103484,[]


In [ ]:
merged['daily_factor'] = merged['statusActs'] + merged['statusFeels']

# uid별로 daily_factor를 합칩니다.
grouped_combined = merged.groupby('uid')['daily_factor'].sum().reset_index()

In [ ]:
grouped_combined

,uid,daily_factor
0,02P89ctRsxaiI9QridPGObRV4Dh1,"[nap, late meal, nap, tired, tired, tired, tir..."
1,034WnkRLrKgC3qGzisY6SBDtztG2,"[alcohol, phone, stretching, phone, alcohol, p..."
2,05qGE7EpymM33qAxi7KxJgwwl0z1,"[study, phone, shower, relaxed, depressed]"
3,0DbVhIqMXiOo6wLjzq8WY2m4FJl2,"[phone, phone, phone, phone, alcohol, late mea..."
4,0OrjreuE1dP9KUkslwK9JgFJ5tI3,"[phone, phone, nap, phone, excited, excited, e..."
...,...,...
1072,zQeN4iM3q4RVqtSiZp5jEqWX9QG2,"[phone, phone, read, tired, tired]"
1073,zcKhOrGj44RZRkf2QHfR3spcC1F3,[]
1074,zidNTzj4NTVFIWHhy4c3CBrR5A82,"[late meal, tv, phone, coffee, nicotine, coffe..."
1075,zmHRVyRKe2MZ1KoPVZLzAqvXVDq2,"[phone, workout, workout, alcohol, stressed, s..."


In [ ]:
aos_sleephour_excel = pd.merge(aos_sleephour_excel, grouped_combined, on = 'uid', how = 'left')

In [ ]:
aos_sleephour_excel['usernameid'] = aos_sleephour_excel['uid']

In [ ]:

aos_sleephour_excel = aos_sleephour_excel[['uid', 'usernameid', 'username', 'average_sleep_hours',
       'average_bedtime', 'average_waketime', 'sleep_pattern',
       'mon_sleep_hours', 'tue_sleep_hours', 'wed_sleep_hours',
       'thu_sleep_hours', 'fri_sleep_hours', 'sat_sleep_hours',
       'sun_sleep_hours', 'mon_waketime', 'tue_waketime', 'wed_waketime',
       'thu_waketime', 'fri_waketime', 'sat_waketime', 'sun_waketime',
       'daily_factor', 'phone_number' ]]


In [ ]:
멈춰 !!

SyntaxError: ignored

In [ ]:
statusActs = aos_sleeptime.groupby('uid')['statusActs'].apply(list).reset_index()

In [ ]:
statusActs

,uid,statusActs
0,02P89ctRsxaiI9QridPGObRV4Dh1,"[[], [], [], [late meal], [nap], [], [nap]]"
1,034WnkRLrKgC3qGzisY6SBDtztG2,"[[alcohol, phone], [alcohol, phone], [nap], [p..."
2,05qGE7EpymM33qAxi7KxJgwwl0z1,"[[study, phone, shower]]"
3,0DbVhIqMXiOo6wLjzq8WY2m4FJl2,"[[phone], [coffee, phone], [alcohol, late meal..."
4,0FwZEU9od7ffGYB0h3lIsau1bMp1,"[[phone], [read]]"
...,...,...
1227,zcKhOrGj44RZRkf2QHfR3spcC1F3,"[[coffee, nicotine], [], [], [], [], [], [], []]"
1228,zdK3HAaUgtNTytU139VfAOw9vqv1,"[[], [nicotine, alcohol, phone]]"
1229,zidNTzj4NTVFIWHhy4c3CBrR5A82,"[[coffee, nicotine], [late meal], [tv, phone],..."
1230,zmHRVyRKe2MZ1KoPVZLzAqvXVDq2,"[[workout], [phone], [alcohol], [], [workout],..."


In [ ]:
from collections import Counter

most_common_labels = []

# 주어진 리스트들을 순회
for label_list in statusActs['statusActs']:
    # 빈 리스트인 경우 "nan"으로 처리
    if not label_list:
        most_common_labels.append('nan')
    else:
        # 모든 리스트의 항목을 하나의 리스트로 펼치기
        all_items = [item for sublist in label_list for item in sublist]

        # 항목별 빈도수 계산
        item_counts = Counter(all_items)

        # 가장 많이 나온 항목 찾기
        if item_counts:
            most_common_item = item_counts.most_common(1)[0][0]
        else:
            most_common_item = 'nan'

        most_common_labels.append(most_common_item)

print(most_common_labels)


['nap', 'phone', 'study', 'late meal', 'phone', 'phone', 'phone', 'phone', 'phone', 'phone', 'shower', 'phone', 'phone', 'phone', 'tv', 'phone', 'study', 'phone', 'phone', 'phone', 'tv', 'nicotine', 'phone', 'phone', 'tv', 'phone', 'nan', 'phone', 'phone', 'shower', 'phone', 'nicotine', 'nicotine', 'phone', 'study', 'phone', 'phone', 'tv', 'phone', 'phone', 'phone', 'tv', 'phone', 'shower', 'phone', 'phone', 'phone', 'read', 'phone', 'tv', 'tv', 'phone', 'nan', 'phone', 'coffee', 'phone', 'phone', 'shower', 'nicotine', 'phone', 'phone', 'phone', 'phone', 'phone', 'nicotine', 'nicotine', 'tv', 'phone', 'phone', 'nicotine', 'phone', 'phone', 'phone', 'study', 'phone', 'nicotine', 'phone', 'phone', 'phone', 'phone', 'nan', 'shower', 'phone', 'tv', 'late meal', 'study', 'late meal', 'phone', 'study', 'nicotine', 'phone', 'coffee', 'phone', 'phone', 'phone', 'phone', 'phone', 'nan', 'phone', 'phone', 'nicotine', 'phone', 'tv', 'alcohol', 'phone', 'coffee', 'phone', 'nan', 'tv', 'tv', 'nicot

In [ ]:
sleepReport = statusActs.copy()
sleepReport.drop('statusActs', axis = 1, inplace = True )
sleepReport['activity'] = most_common_labels
statusFeels = aos_sleeptime.groupby('uid')['statusFeels'].apply(list).reset_index()
from collections import Counter

most_common_labels = []

# 주어진 리스트들을 순회
for label_list in statusFeels['statusFeels']:
    # 빈 리스트인 경우 "nan"으로 처리
    if not label_list:
        most_common_labels.append('nan')
    else:
        # 모든 리스트의 항목을 하나의 리스트로 펼치기
        all_items = [item for sublist in label_list for item in sublist]

        # 항목별 빈도수 계산
        item_counts = Counter(all_items)

        # 가장 많이 나온 항목 찾기
        if item_counts:
            most_common_item = item_counts.most_common(1)[0][0]
        else:
            most_common_item = 'nan'

        most_common_labels.append(most_common_item)

print(most_common_labels)
sleepReport['emotion'] = most_common_labels


['tired', 'tired', 'relaxed', 'tired', 'nervous', 'excited', 'tired', 'relaxed', 'tired', 'stressed', 'anxious', 'stressed', 'nervous', 'tired', 'anxious', 'depressed', 'tired', 'tired', 'stressed', 'anxious', 'tired', 'tired', 'tired', 'tired', 'tired', 'tired', 'nan', 'tired', 'excited', 'tired', 'tired', 'tired', 'excited', 'relaxed', 'tired', 'tired', 'tired', 'relaxed', 'nervous', 'relaxed', 'tired', 'relaxed', 'tired', 'excited', 'excited', 'tired', 'tired', 'excited', 'stressed', 'tired', 'tired', 'tired', 'tired', 'tired', 'tired', 'depressed', 'tired', 'tired', 'tired', 'anxious', 'nan', 'stressed', 'tired', 'stressed', 'tired', 'tired', 'tired', 'tired', 'tired', 'nan', 'relaxed', 'tired', 'tired', 'tired', 'stressed', 'tired', 'relaxed', 'tired', 'tired', 'tired', 'relaxed', 'tired', 'nervous', 'depressed', 'tired', 'stressed', 'stressed', 'anxious', 'tired', 'stressed', 'excited', 'stressed', 'stressed', 'relaxed', 'relaxed', 'tired', 'stressed', 'nan', 'tired', 'tired', 't

In [ ]:
sleepReport

,uid,activity,emotion
0,02P89ctRsxaiI9QridPGObRV4Dh1,nap,tired
1,034WnkRLrKgC3qGzisY6SBDtztG2,phone,tired
2,05qGE7EpymM33qAxi7KxJgwwl0z1,study,relaxed
3,0DbVhIqMXiOo6wLjzq8WY2m4FJl2,phone,nervous
4,0OrjreuE1dP9KUkslwK9JgFJ5tI3,phone,excited
...,...,...,...
1072,zQeN4iM3q4RVqtSiZp5jEqWX9QG2,phone,tired
1073,zcKhOrGj44RZRkf2QHfR3spcC1F3,nan,nan
1074,zidNTzj4NTVFIWHhy4c3CBrR5A82,coffee,stressed
1075,zmHRVyRKe2MZ1KoPVZLzAqvXVDq2,workout,tired


In [ ]:
aos_sleephour_excel = pd.merge(aos_sleephour_excel,sleepReport,on='uid', how='left')

In [ ]:
aos_prev = pd.read_csv("/content/aos_report1623.csv", index_col = False)


In [ ]:
aos_prev = aos_prev[['uid','average_sleep_hours',
       'average_bedtime', 'average_waketime' ]]
aos_prev.columns = ['uid','prev_average_sleep_hours',
       'prev_bedtime', 'prev_waketime' ]
aos_sleephour_excel = pd.merge(aos_sleephour_excel, aos_prev, on = 'uid', how = 'left')


In [ ]:
aos_sleephour_excel.to_csv("aos_report2329.csv", index = False, encoding = 'utf-8-sig')

In [ ]:
멈춰!

In [ ]:
aos_sleeptime_091824['comment'] = aos_sleeptime_091824['sleepComment']

In [ ]:
comment_pivot = aos_sleeptime_091824.pivot_table(index='uid', columns='modified_start_weekday', values='comment', aggfunc='last')[weekday_kor2].reset_index()

In [ ]:
aos_sleephour_excel = pd.merge(aos_sleephour_excel, comment_pivot, on='uid', how='left')


In [ ]:
aos_sleephour_excel

In [ ]:
aos_sleephour_excel